## Reference

https://stackoverflow.com/questions/33759623/tensorflow-how-to-save-restore-a-model

http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/

In [48]:
import tensorflow as tf
import os

## Meta Graph

> This is a protocol buffer which saves the complete Tensorflow graph; i.e. all variables, operations, collections etc. 

> This file has **.meta** extension.

## Checkpoint file

> This is a binary file which contains all the values of the weights, biases, gradients and all the other variables saved. 

> This file has an extension **.ckpt**.

## Save

In [49]:
model_save_path = '/tmp/my_tf_test_model'

### Tensorflow variables are only alive in session, you have to save them in session

In [50]:
tf.reset_default_graph()

w1 = tf.Variable(tf.truncated_normal(shape=[10]), name='w1')
w2 = tf.Variable(tf.truncated_normal(shape=[20]), name='w2')

tf.add_to_collection('vars', w1)
tf.add_to_collection('vars', w2)

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, model_save_path)   
    
    # Training...
    # subsequent trainings meta graph is not updated so you don't need to resave it again
    # set write_meta_graph=False
    # saver.save(sess, model_save_path, global_step=step, write_meta_graph=False)
    
    # other parameters to tune
    # saver.save(sess, model_save_path, global_step=step, 
    #            write_meta_graph=False,
    #            max_to_keep=4,
    #            keep_checkpoint_every_n_hours=2)
    

In [51]:
% ls -al /tmp/my*

-rw-r--r-- 1 root root    4 Sep  1 08:18 /tmp/my_test_model-1000.data-00000-of-00001
-rw-r--r-- 1 root root  120 Sep  1 08:18 /tmp/my_test_model-1000.index
-rw-r--r-- 1 root root 2790 Sep  1 08:18 /tmp/my_test_model-1000.meta
-rw-r--r-- 1 root root    4 Sep  1 08:21 /tmp/my_tf_test_model-1000.data-00000-of-00001
-rw-r--r-- 1 root root  120 Sep  1 08:21 /tmp/my_tf_test_model-1000.index
-rw-r--r-- 1 root root 2790 Sep  1 08:21 /tmp/my_tf_test_model-1000.meta
-rw-r--r-- 1 root root  120 Sep  1 08:23 /tmp/my_tf_test_model.data-00000-of-00001
-rw-r--r-- 1 root root  143 Sep  1 08:23 /tmp/my_tf_test_model.index
-rw-r--r-- 1 root root 4817 Sep  1 08:23 /tmp/my_tf_test_model.meta


## Save only some variables

```Python
w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.Variable(tf.random_normal(shape=[5]), name='w2')

# only save w1 and w2
saver = tf.train.Saver([w1, w2])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, '/tmp/my_test_model', global_step=1000)
```

## Load

In [52]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('{}.meta'.format(model_save_path))
    new_saver.restore(sess, tf.train.latest_checkpoint(os.path.dirname(model_save_path)))
    
    all_vars = tf.get_collection('vars')

    for v in all_vars:
        v_ = sess.run(v)
        print(v_)

INFO:tensorflow:Restoring parameters from /tmp/my_tf_test_model
[-1.43592906  0.49199969  0.3102732   0.70848399 -0.28851402 -0.91920251
  0.67731011  1.16958964 -0.35451865 -0.31641874]
[ 0.95022619  0.79574502 -0.40759259  0.03828121 -0.47309098 -1.92819595
  0.07515428  0.06131484 -0.42148468 -0.19982119  0.59885925 -1.40948188
 -0.45097643 -0.43091989 -0.51169211 -0.12227061  0.28253883 -1.29828274
 -0.71952969 -0.20967431]
[-1.43592906  0.49199969  0.3102732   0.70848399 -0.28851402 -0.91920251
  0.67731011  1.16958964 -0.35451865 -0.31641874]
[ 0.95022619  0.79574502 -0.40759259  0.03828121 -0.47309098 -1.92819595
  0.07515428  0.06131484 -0.42148468 -0.19982119  0.59885925 -1.40948188
 -0.45097643 -0.43091989 -0.51169211 -0.12227061  0.28253883 -1.29828274
 -0.71952969 -0.20967431]


# Full Case

### Save Graph & Weights

In [53]:
import tensorflow as tf

tf.reset_default_graph()

# Prepare to feed input, i.e. feed_dict and placeholders
w1 = tf.placeholder("float", name="w1")
w2 = tf.placeholder("float", name="w2")
b1 = tf.Variable(2.0, name="bias")
feed_dict ={w1:4, w2:8}

# Define a test operation that we will restore
# (w1 + w2) * b1
w3 = tf.add(w1, w2)
w4 = tf.multiply(w3, b1, name="op_to_restore")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Create a saver object which will save all the variables
    saver = tf.train.Saver()

    # Run the operation by feeding input
    print(sess.run(w4, feed_dict))
    # Prints 24 which is sum of (w1+w2)*b1 

    #Now, save the graph
    saver.save(sess, model_save_path, global_step=1000)

24.0


In [54]:
% ls $model_save_path*

/tmp/my_tf_test_model-1000.data-00000-of-00001
/tmp/my_tf_test_model-1000.index
/tmp/my_tf_test_model-1000.meta
/tmp/my_tf_test_model.data-00000-of-00001
/tmp/my_tf_test_model.index
/tmp/my_tf_test_model.meta


### Get specific weight or operator

In [55]:
graph = tf.get_default_graph()

# How to access saved variable/Tensor/placeholders 
w1 = graph.get_tensor_by_name("w1:0")
print(w1.name)

## How to access saved operation
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")
print(op_to_restore.name)

w1:0
op_to_restore:0


### Load trained model, feed new data for prediction

#### Note the feed_dict's key is the Actural Tensor

In [56]:
import tensorflow as tf

with tf.Session() as sess:
    #First let's load meta graph and restore weights
    saver = tf.train.import_meta_graph('{}-1000.meta'.format(model_save_path))
    saver.restore(sess, tf.train.latest_checkpoint(os.path.dirname(model_save_path)))

    # Now, let's access and create placeholders variables and
    # create feed-dict to feed new data
    graph = tf.get_default_graph()
    
    w1 = graph.get_tensor_by_name("w1:0")
    w2 = graph.get_tensor_by_name("w2:0")
    
    # new data to feed: predict
    feed_dict ={w1:13.0, w2:17.0}

    #Now, access the op that you want to run. 
    op_to_restore = graph.get_tensor_by_name("op_to_restore:0")

    print(sess.run(op_to_restore, feed_dict))


INFO:tensorflow:Restoring parameters from /tmp/my_tf_test_model-1000
60.0


### Load Graph, add more Layers

In [57]:
import tensorflow as tf

with tf.Session() as sess:
    # First let's load meta graph and restore weights
    saver = tf.train.import_meta_graph('{}-1000.meta'.format(model_save_path))
    saver.restore(sess, tf.train.latest_checkpoint(os.path.dirname(model_save_path)))

    # Now, let's access and create placeholders variables and
    # create feed-dict to feed new data
    graph = tf.get_default_graph()
    w1 = graph.get_tensor_by_name("w1:0")
    w2 = graph.get_tensor_by_name("w2:0")
    
    feed_dict ={w1:13.0, w2:17.0}

    # Now, access the op that you want to run. 
    op_to_restore = graph.get_tensor_by_name("op_to_restore:0")

    # Add more to the current graph
    add_on_op = tf.multiply(op_to_restore, 2)

    print(sess.run(add_on_op, feed_dict))
    # This will print 120.

INFO:tensorflow:Restoring parameters from /tmp/my_tf_test_model-1000
120.0


# Load part of pre-trained Graph and rewire it

```Python
saver = tf.train.import_meta_graph('vgg.meta')
# Access the graph
graph = tf.get_default_graph()
## Prepare the feed_dict for feeding data for fine-tuning 

#Access the appropriate output for fine-tuning
fc7= graph.get_tensor_by_name('fc7:0')

#use this if you only want to change gradients of the last layer
fc7 = tf.stop_gradient(fc7) # It's an identity function
fc7_shape= fc7.get_shape().as_list()

new_outputs = 2
weights = tf.Variable(tf.truncated_normal([fc7_shape[3], num_outputs], stddev=0.05))
biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
output = tf.matmul(fc7, weights) + biases
pred = tf.nn.softmax(output)

# Now, you run this with fine-tuning data in sess.run()
```